<a href="https://colab.research.google.com/github/AyushXRane/Alzheimer-MRI-Classifier/blob/main/Alzheimer's_CNN_augmented_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Project/Alzheimers.zip'

Streaming output truncated to the last 5000 lines.
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_102.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_103.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_104.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_105.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_106.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_107.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_108.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_109.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_110.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_111.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_112.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_113.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_114.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_115.jpg  
  inflating

In [ ]:
'''NEED TO FIX TRAIN/TEST SPLIT, FIX CLASS IMBALANCE, CROSS VALIDATION'''

In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
!ls './Data/'



ls: cannot access './Data/': No such file or directory


In [ ]:
import os
file_path = './Data/'

files = os.listdir(file_path)
print(files)



['Moderate Dementia', 'Non Demented', 'Very mild Dementia', 'Mild Dementia']


In [ ]:
import os
from collections import defaultdict

def count_images_per_patient_for_all_classes(parent_folder):
    patient_image_counts = defaultdict(int)

    for class_folder in os.listdir(parent_folder):
        class_path = os.path.join(parent_folder, class_folder)

        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    try:
                        patient_id = filename.split('_')[1]
                    except IndexError:
                        continue

                    patient_image_counts[patient_id] += 1

    for patient_id, count in patient_image_counts.items():
        print(f"Patient ID: {patient_id}, Number of images: {count}")

    total_patients = len(patient_image_counts)
    print(f"Total number of unique patients: {total_patients}")

parent_folder = './Data'
count_images_per_patient_for_all_classes(parent_folder)


Patient ID: 0084, Number of images: 244
Patient ID: 0161, Number of images: 244
Patient ID: 0380, Number of images: 244
Patient ID: 0312, Number of images: 244
Patient ID: 0298, Number of images: 244
Patient ID: 0272, Number of images: 244
Patient ID: 0217, Number of images: 244
Patient ID: 0042, Number of images: 244
Patient ID: 0267, Number of images: 183
Patient ID: 0263, Number of images: 244
Patient ID: 0233, Number of images: 244
Patient ID: 0247, Number of images: 244
Patient ID: 0015, Number of images: 183
Patient ID: 0003, Number of images: 244
Patient ID: 0041, Number of images: 244
Patient ID: 0060, Number of images: 244
Patient ID: 0143, Number of images: 244
Patient ID: 0374, Number of images: 244
Patient ID: 0098, Number of images: 244
Patient ID: 0352, Number of images: 183
Patient ID: 0158, Number of images: 244
Patient ID: 0094, Number of images: 244
Patient ID: 0288, Number of images: 244
Patient ID: 0166, Number of images: 244
Patient ID: 0304, Number of images: 244


In [ ]:
import os
import random

# Define the paths to the two class folders
classes_to_reduce = {
    "Class1": "/content/Data/Non Demented",  # Replace with your actual folder path
    "Class2": "/content/Data/Very mild Dementia",
    # Replace with your actual folder path
}

# Specify the target number of images to keep in each class
target_size = 5000  # Adjust this to the desired number of images to keep

# Iterate over each class
for class_name, folder_path in classes_to_reduce.items():
    # List all files in the class folder
    files = os.listdir(folder_path)
    num_files = len(files)
    print(f"{class_name}: {num_files} images found.")

    # Check if the class has more images than the target size
    if num_files > target_size:
        # Calculate how many files to remove
        excess = num_files - target_size
        print(f"Removing {excess} files from {class_name}...")

        # Randomly select files to delete
        files_to_delete = random.sample(files, excess)

        # Remove the selected files
        for file_name in files_to_delete:
            file_path = os.path.join(folder_path, file_name)
            os.remove(file_path)

        print(f"Finished removing {excess} images from {class_name}. Remaining files: {len(os.listdir(folder_path))}")
    else:
        print(f"No images removed from {class_name}; already meets target size.")


Class1: 67222 images found.
Removing 62222 files from Class1...
Finished removing 62222 images from Class1. Remaining files: 5000
Class2: 13725 images found.
Removing 8725 files from Class2...
Finished removing 8725 images from Class2. Remaining files: 5000


In [ ]:
print({len(os.listdir('./Data/Non Demented'))})
print({len(os.listdir('./Data/Mild Dementia'))})
print({len(os.listdir('./Data/Moderate Dementia'))})
print({len(os.listdir('./Data/Very mild Dementia'))})


{5000}
{5002}
{488}
{5000}


In [ ]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder

# Function to load images from folder
def load_images_from_folder(folder, category):
    images = []
    labels = []
    if not os.path.exists(folder):
        raise ValueError(f"Directory does not exist: {folder}")
    print(f"Loading images from {folder}...")
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (150, 150))  # Resize images
            img = img.astype(np.float32) / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(category)  # Use the passed category as label
    if not images or not labels:
        raise ValueError("No images or labels found in the specified directory.")
    return np.array(images), np.array(labels)

# Load images from each directory
mild_demented_dir = './Data/Mild Dementia'
moderate_demented_dir = './Data/Moderate Dementia'
non_demented_dir = './Data/Non Demented'
very_mild_demented_dir = './Data/Very mild Dementia'

# Load the images from each directory with corresponding labels
mild_demented_data, mild_demented_labels = load_images_from_folder(mild_demented_dir, 'mild')
moderate_demented_data, moderate_demented_labels = load_images_from_folder(moderate_demented_dir, 'moderate')
non_demented_data, non_demented_labels = load_images_from_folder(non_demented_dir, 'non')
very_mild_demented_data, very_mild_demented_labels = load_images_from_folder(very_mild_demented_dir, 'very_mild')

# Concatenate data and labels
data = np.concatenate((mild_demented_data, moderate_demented_data, non_demented_data, very_mild_demented_data))
labels = np.concatenate((mild_demented_labels, moderate_demented_labels, non_demented_labels, very_mild_demented_labels))



# Apply Data Augmentation for "moderate" class
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,  # Less shift
    height_shift_range=0.1,
    shear_range=0.1,  # Less shear
    zoom_range=0.1,  # Less zoom
    horizontal_flip=False,  # Keep off unless data allows
    fill_mode='nearest'  # Avoid artifacts
)



# Augment moderate class images
augmented_images = []
# Loop through each image in the "moderate" class
for i in range(len(moderate_demented_data)):
    img = moderate_demented_data[i].reshape((1, *moderate_demented_data[i].shape))  # Reshape to 4D for flow
    # Apply augmentation for the current image
    for batch in datagen.flow(img, batch_size=1):
        augmented_images.append(batch[0])
        if len(augmented_images) >= 4512:  # We want 4512 new augmented images
            break
    if len(augmented_images) >= 4512:
        break


# Concatenate the augmented images with the original data
data = np.concatenate((data, np.array(augmented_images)))

# Update labels to match the augmented images
labels = np.concatenate((labels, np.full(len(augmented_images), 'moderate')))

# One-Hot Encoding the updated labels
one_hot_encoder = OneHotEncoder(sparse_output=False)
labels_encoded = one_hot_encoder.fit_transform(labels.reshape(-1, 1))

# Verify data and labels shapes
print("Updated data shape:", data.shape)
print("Updated labels shape:", labels_encoded.shape)


Loading images from ./Data/Mild Dementia...
Loading images from ./Data/Moderate Dementia...
Loading images from ./Data/Non Demented...
Loading images from ./Data/Very mild Dementia...
Updated data shape: (20002, 150, 150, 3)
Updated labels shape: (20002, 4)


In [ ]:
print(np.unique(labels))

['mild' 'moderate' 'non' 'very_mild']


In [ ]:
print("Encoded Labels:", labels_encoded)
print("Unique Encoded Values:", np.unique(labels_encoded))

Encoded Labels: [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]
Unique Encoded Values: [0. 1.]


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)



In [ ]:
from keras.models import Sequential
from keras import layers


model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])

#batch normalization after each CV layer
#droupout after each maxpooling, ex model.add(Dropout(0.25))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last 10 layers
for layer in base_model.layers[-15:]:
    layer.trainable = True


model_preTrained = Sequential([
    base_model,
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  # Output layer for 4 classes
])






94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

#Train the model
history = model.fit(train_data, train_labels, validation_split=0.2, epochs=10, batch_size=32)

#Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_accuracy)

from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.argmax(test_labels, axis=1)
report = classification_report(true_classes, predicted_classes, target_names=one_hot_encoder.categories_[0])
print("Classification Report:")
print(report)

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 231s 570ms/step - accuracy: 0.5374 - loss: 1.2340 - val_accuracy: 0.4958 - val_loss: 3.5935
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 214s 535ms/step - accuracy: 0.8256 - loss: 0.4702 - val_accuracy: 0.7688 - val_loss: 0.5605
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 214s 536ms/step - accuracy: 0.8970 - loss: 0.2980 - val_accuracy: 0.8850 - val_loss: 0.3083
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 221s 554ms/step - accuracy: 0.9337 - loss: 0.1959 - val_accuracy: 0.9425 - val_loss: 0.2032
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 218s 544ms/step - accuracy: 0.9591 - loss: 0.1280 - val_accuracy: 0.9494 - val_loss: 0.1672
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 229s 573ms/step - accuracy: 0.9761 - loss: 0.0874 - val_accuracy: 0.9722 - val_loss: 0.0947
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 228s 570ms/step - accuracy: 0.9828 - loss: 0.0578 - val_accuracy: 0.9691 - val_loss: 0.1026
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 233s 582ms/step - accuracy: 0.9924 -

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np

def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  ])

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(data)):
    print(f"Training on fold {fold + 1}...")

    X_train, X_val = data[train_index], data[val_index]
    y_train, y_val = labels_encoded[train_index], labels_encoded[val_index]

    model = create_model()

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

print(f"\nCross-Validation Accuracies: {fold_accuracies}")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Standard Deviation: {np.std(fold_accuracies):.4f}")


Training on fold 1...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 285s 563ms/step - accuracy: 0.5952 - loss: 1.1516 - val_accuracy: 0.3439 - val_loss: 3.5049
Epoch 2/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 277s 553ms/step - accuracy: 0.8614 - loss: 0.3811 - val_accuracy: 0.8135 - val_loss: 0.4256
Epoch 3/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 288s 575ms/step - accuracy: 0.9271 - loss: 0.2123 - val_accuracy: 0.9228 - val_loss: 0.2056
Epoch 4/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 279s 557ms/step - accuracy: 0.9613 - loss: 0.1262 - val_accuracy: 0.9613 - val_loss: 0.1226
Epoch 5/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 304s 606ms/step - accuracy: 0.9763 - loss: 0.0828 - val_accuracy: 0.9450 - val_loss: 0.1505
Fold 1 Accuracy: 0.9450
Training on fold 2...
Epoch 1/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 288s 571ms/step - accuracy: 0.6268 - loss: 1.0945 - val_accuracy: 0.4856 - val_loss: 1.6844
Epoch 2/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 284s 568ms/step - accuracy: 0.8631 - loss: 0.3546 - val_accuracy: 0.8420 - val_loss: 0.3579
Epoch 3/5
501/501 ━━━━━━━━━━━━━━━━━━━

In [ ]:

model_preTrained.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

history_preTrained = model_preTrained.fit(train_data,
                    train_labels,
                    epochs=10,
                    batch_size=32,
                    validation_split = 0.2)
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_accuracy)


from sklearn.metrics import classification_report, confusion_matrix
predictions = model_preTrained.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.argmax(test_labels, axis=1)
report = classification_report(true_classes, predicted_classes, target_names=one_hot_encoder.categories_[0])
print("Classification Report:")
print(report)

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 396s 972ms/step - accuracy: 0.6101 - loss: 1.0555 - val_accuracy: 0.7748 - val_loss: 0.5724
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 434s 1s/step - accuracy: 0.7743 - loss: 0.5246 - val_accuracy: 0.8394 - val_loss: 0.3860
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 373s 932ms/step - accuracy: 0.8421 - loss: 0.3813 - val_accuracy: 0.8710 - val_loss: 0.3132
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 367s 918ms/step - accuracy: 0.8837 - loss: 0.2922 - val_accuracy: 0.8947 - val_loss: 0.2599
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 381s 953ms/step - accuracy: 0.8979 - loss: 0.2506 - val_accuracy: 0.8935 - val_loss: 0.2776
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 416s 1s/step - accuracy: 0.9176 - loss: 0.2111 - val_accuracy: 0.9285 - val_loss: 0.1873
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 362s 906ms/step - accuracy: 0.9285 - loss: 0.1844 - val_accuracy: 0.9485 - val_loss: 0.1383
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 379s 947ms/step - accuracy: 0.9445 - loss:

NameError: name 'model' is not defined

In [ ]:
test_loss, test_accuracy = model_preTrained.evaluate(test_data, test_labels)
print("Test accuracy:", test_accuracy)


from sklearn.metrics import classification_report, confusion_matrix
predictions = model_preTrained.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.argmax(test_labels, axis=1)
report = classification_report(true_classes, predicted_classes, target_names=one_hot_encoder.categories_[0])
print("Classification Report:")
print(report)

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

126/126 ━━━━━━━━━━━━━━━━━━━━ 74s 589ms/step - accuracy: 0.9485 - loss: 0.1498
Test accuracy: 0.9475131034851074
126/126 ━━━━━━━━━━━━━━━━━━━━ 90s 704ms/step
Classification Report:
              precision    recall  f1-score   support

        mild       0.97      0.99      0.98       995
    moderate       0.99      1.00      0.99       975
         non       0.90      0.90      0.90      1021
   very_mild       0.93      0.90      0.91      1010

    accuracy                           0.95      4001
   macro avg       0.95      0.95      0.95      4001
weighted avg       0.95      0.95      0.95      4001

Confusion Matrix:
[[984   0   6   5]
 [  0 975   0   0]
 [ 26   8 924  63]
 [  8   3  91 908]]


In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
import numpy as np

# Load the ResNet50 base model (excluding the top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last 15 layers for fine-tuning
for layer in base_model.layers[-15:]:
    layer.trainable = True

# Function to create the model
def create_model():
    model = Sequential([
        base_model,
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  # Output layer for 4 classes
    ])

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(data)):
    print(f"Training on fold {fold + 1}...")

    X_train, X_val = data[train_index], data[val_index]
    y_train, y_val = labels_encoded[train_index], labels_encoded[val_index]

    model = create_model()

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

print(f"\nCross-Validation Accuracies: {fold_accuracies}")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Standard Deviation: {np.std(fold_accuracies):.4f}")


Training on fold 1...
Epoch 1/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 552s 1s/step - accuracy: 0.6258 - loss: 0.9412 - val_accuracy: 0.7933 - val_loss: 0.4867
Epoch 2/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 523s 1s/step - accuracy: 0.7884 - loss: 0.5136 - val_accuracy: 0.8430 - val_loss: 0.3841
Epoch 3/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 521s 1s/step - accuracy: 0.8672 - loss: 0.3325 - val_accuracy: 0.7821 - val_loss: 0.5915
Epoch 4/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 587s 1s/step - accuracy: 0.8665 - loss: 0.3260 - val_accuracy: 0.8178 - val_loss: 0.4925
Epoch 5/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 501s 1s/step - accuracy: 0.8828 - loss: 0.2987 - val_accuracy: 0.8793 - val_loss: 0.2860
Fold 1 Accuracy: 0.8793
Training on fold 2...
Epoch 1/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 534s 1s/step - accuracy: 0.8137 - loss: 0.4961 - val_accuracy: 0.9145 - val_loss: 0.2192
Epoch 2/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 500s 998ms/step - accuracy: 0.8924 - loss: 0.2683 - val_accuracy: 0.9018 - val_loss: 0.2456
Epoch 3/5
501/501 ━━━━━━━━━━━━━━━

In [ ]:
'''model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

#Train the model
history = model.fit(train_data, train_labels, validation_split=0.2, epochs=10, batch_size=32)

#Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_accuracy)'''

In [ ]:
'''
model_preTrained.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

history_preTrained = model_preTrained.fit(train_data,
                    train_labels,
                    epochs=10,
                    batch_size=32,
                    validation_split = 0.2)
                    '''

In [ ]:
'''rom sklearn.metrics import classification_report, confusion_matrix
predictions = model_preTrained.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.argmax(test_labels, axis=1)
report = classification_report(true_classes, predicted_classes, target_names=one_hot_encoder.categories_[0])
print("Classification Report:")
print(report)

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)'''

In [ ]:
'''from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np

def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  ])

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(data)):
    print(f"Training on fold {fold + 1}...")

    X_train, X_val = data[train_index], data[val_index]
    y_train, y_val = labels_encoded[train_index], labels_encoded[val_index]

    model = create_model()

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

print(f"\nCross-Validation Accuracies: {fold_accuracies}")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Standard Deviation: {np.std(fold_accuracies):.4f}")'''
